# Controllers

To actively manipulate the computing or energy systems during experiments, we need to implement a custom `Controller`.

In this example, our controller will adjust the power consumption of the compute nodes depending on the current power delta. 

In [2]:
from __future__ import annotations
from vessim.actor import ComputingSystem, Generator
from vessim.controller import Controller, Monitor
from vessim.cosim import Environment
from vessim.power_meter import MockPowerMeter
from vessim.signal import HistoricalSignal
from vessim.storage import SimpleBattery

# Hotfix to execute asyncio in Jupyter
import nest_asyncio
nest_asyncio.apply()

The most important aspect of this procedure is the implementation of the
`step()` method, which needs to be implemented as dictated by the Controller
ABC. The `step()` method is called every simulation step and allows a Controller
to act on the power delta from the Microgrid, the current simulation time and
the activity of the Actors. The `SimpleLoadBalancingController` only utilizes
the power delta and adjusts, depending on this value, the power consumption of
the computing system and in turn the power delta in the next simulation step.

In [3]:
class SimpleLoadBalancingController(Controller):
    def __init__(self, max_load_adjustment: float, power_meters: list[MockPowerMeter]):
        super().__init__()
        self.max_load_adjustment = max_load_adjustment
        self.power_meters = power_meters

    def start(self, microgrid) -> None:
        pass

    def step(self, time: int, p_delta: float, actor_infos: dict) -> None:
        # Calculate the maximum adjustment per MockPowerMeter
        adjustment_per_meter = min(abs(p_delta), self.max_load_adjustment) / len(self.power_meters)

        # Adjust the power setpoint for each MockPowerMeter
        for power_meter in self.power_meters:
            current_power = power_meter.measure()
            # Determine direction of adjustment
            if p_delta < 0:
                new_power = current_power + adjustment_per_meter
            else:
                new_power = max(0, current_power - adjustment_per_meter)
            power_meter.set_power(new_power)

Now we can add the controller to the basic scenario by instantiating it with a
reference to the power meters it can control and the maximum load adjustment for
each step. The rest of the scenario remains unchanged.

In [4]:
environment = Environment(sim_start="2020-06-11 00:00:00")

power_meters: list = [
    MockPowerMeter(p=3),
    MockPowerMeter(p=7),
]
monitor = Monitor()  # Stores simulation result on each step
load_balancer = SimpleLoadBalancingController(max_load_adjustment=2, power_meters=power_meters)
environment.add_microgrid(
    actors=[
        ComputingSystem(power_meters=power_meters),
        Generator(signal=HistoricalSignal.from_dataset("solcast2022_global"), column="Berlin"),
    ],
    storage=SimpleBattery(capacity=1000, charge_level=500),
    controllers=[monitor, load_balancer],
    step_size=60,  # Global step size (can be overridden by actors or controllers)
)

environment.run(until=24 * 3600) # 24h
monitor.to_csv("result_csv")

2024-03-15 11:45:05.402 | INFO     | mosaik.scenario:start:280 - Starting "Actor" as "Actor-0" ...
2024-03-15 11:45:05.403 | INFO     | mosaik.scenario:start:280 - Starting "Actor" as "Actor-1" ...
2024-03-15 11:45:05.403 | INFO     | mosaik.scenario:start:280 - Starting "Grid" as "Grid-0" ...
2024-03-15 11:45:05.404 | INFO     | mosaik.scenario:start:280 - Starting "Controller" as "Controller-0" ...
2024-03-15 11:45:05.405 | INFO     | mosaik.scenario:start:280 - Starting "Controller" as "Controller-1" ...
2024-03-15 11:45:05.406 | INFO     | mosaik.scenario:run:598 - Starting simulation.
100%|██████████| 86400/86400 [00:00<00:00, 156784.40steps/s]
2024-03-15 11:45:05.961 | INFO     | mosaik.scenario:run:646 - Simulation finished successfully.
